# Explication

* Le code génère des données fictives de transactions bancaires pour simuler les habitudes de dépenses et de revenus d'un individu.

* Il utilise la bibliothèque Faker pour créer 500 utilisateurs fictifs et génère 10 000 transactions fictives pour ces utilisateurs, y compris des informations telles que la date, le montant de la transaction, la catégorie, et le solde.

* Les données fictives sont stockées dans un DataFrame appelé "df", avec des colonnes telles que "Utilisateur", "Date", "Day", "Category", "Credit Amount (DZD)", et "Balance".

* Les données sont prétraitées, avec une séparation en caractéristiques (X) et une variable cible (y), où X comprend les colonnes "Day", "Category", et "Balance", et y comprend la colonne "Credit Amount (DZD)" à prédire.

* Un modèle est créé et entraîné avec les données X et y.

* Le modèle est utilisé pour faire des prédictions sur les dépenses futures en supposant que la catégorie et le solde restent les mêmes que la dernière transaction enregistrée.

* Les prédictions sont affichées, montrant la date et le montant de crédit prévu pour les prochains jours.

* Ce code permet de générer des prédictions de flux de trésorerie basées sur des données fictives de transactions bancaires et un modèle de régression linéaire.

In [ ]:
pip install faker

In [ ]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker('fr_FR')

donnees = []

utilisateurs = []
for _ in range(500):
    nom = fake.first_name()
    prenom = fake.last_name()
    utilisateurs.append(f"{nom} {prenom}")

categories = ['Shopping', 'Groceries', 'Entertainment', 'Salary', 'Transfer']

for _ in range(10000):
    utilisateur = random.choice(utilisateurs)
    date_transaction = fake.date_between_dates(date_start=datetime.today().replace(month=1, day=1), date_end=datetime.today().replace(month=12, day=31))
    montant_transaction = fake.random_int(min=1000, max=10000)
    balance = fake.random_int(min=28000, max=80000)
    category = random.choice(categories)

    donnees.append([utilisateur, date_transaction, date_transaction.strftime("%A"), category, montant_transaction, balance])

df = pd.DataFrame(donnees, columns=["Utilisateur", "Date", "Day", "Category", "Credit Amount (DZD)", "Balance"])

df.to_csv('releve_bancaire_categories.csv', index=False)


In [ ]:
df

,Utilisateur,Date,Day,Category,Credit Amount (DZD),Balance
0,Christophe Perrin,2023-09-05,Tuesday,Shopping,8903,55837
1,Sébastien Lambert,2023-06-06,Tuesday,Salary,9776,66168
2,Thibaut Boyer,2023-01-02,Monday,Transfer,7718,28696
3,Raymond Torres,2023-10-28,Saturday,Shopping,5694,56011
4,Anaïs Renard,2023-04-02,Sunday,Entertainment,6638,65166
...,...,...,...,...,...,...
9995,Élisabeth Girard,2023-07-25,Tuesday,Entertainment,4206,75323
9996,Zacharie Loiseau,2023-09-07,Thursday,Salary,1161,49419
9997,Henri Gauthier,2023-07-16,Sunday,Groceries,2524,30677
9998,Michelle Lucas,2023-01-08,Sunday,Transfer,8541,32219


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Charger le jeu de données
df = pd.read_csv('releve_bancaire_categories.csv')

# Préparer les données pour la prédiction
X = df[['Day', 'Category', 'Balance']]
y = df['Credit Amount (DZD)']

# Créer un pré-processeur pour gérer les caractéristiques catégorielles et numériques
categorical_features = ['Day', 'Category']
numeric_features = ['Balance']

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Créer et entraîner un modèle de régression linéaire
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', LinearRegression())])

model.fit(X, y)

# Prévoir les dépenses pour les prochains jours
n_days_to_predict = 20  # Par exemple, pour prédire les 20 prochains jours du mois
last_date = pd.to_datetime(df['Date'].max())

predictions = []
for day in range(1, n_days_to_predict + 1):
    next_date = last_date + pd.DateOffset(days=day)
    day_of_week = next_date.strftime('%A')
    # Supposons que la catégorie et le solde restent les mêmes pour les prédictions
    category = df['Category'].iloc[-1]
    balance = df['Balance'].iloc[-1]

    # Préparer les données pour la prédiction
    prediction_data = pd.DataFrame({'Day': [day_of_week], 'Category': [category], 'Balance': [balance]})

    # Faire la prédiction
    prediction = model.predict(prediction_data)
    predictions.append({
        'Date': next_date,
        'Predicted Credit Amount (DZD)': prediction[0]
    })

# Convertir les prédictions en DataFrame
predictions_df = pd.DataFrame(predictions)

# Afficher les prédictions
print(predictions_df)


         Date  Predicted Credit Amount (DZD)
0  2024-01-01                    5408.481928
1  2024-01-02                    5510.992314
2  2024-01-03                    5473.739113
3  2024-01-04                    5511.017694
4  2024-01-05                    5508.727457
5  2024-01-06                    5442.179898
6  2024-01-07                    5508.019692
7  2024-01-08                    5408.481928
8  2024-01-09                    5510.992314
9  2024-01-10                    5473.739113
10 2024-01-11                    5511.017694
11 2024-01-12                    5508.727457
12 2024-01-13                    5442.179898
13 2024-01-14                    5508.019692
14 2024-01-15                    5408.481928
15 2024-01-16                    5510.992314
16 2024-01-17                    5473.739113
17 2024-01-18                    5511.017694
18 2024-01-19                    5508.727457
19 2024-01-20                    5442.179898
